In [ ]:
import os
import json
import random
import gc
from datetime import datetime
from tqdm import tqdm
from llama_cpp import Llama
from datasets import load_dataset
import pandas as pd

# ========== Task Configurations ==========
TASKS = {
    "BoolQ": {
        "dataset": "boolq",
        "split": "validation",
        "answer_type": "bool"
    },
    "PIQA": {
        "dataset": "piqa",
        "split": "validation",
        "answer_type": "mc",
        "choices": ["A", "B"]
    },
    "OpenBookQA": {
        "dataset": "openbookqa",
        "split": "test",
        "subset": "main",
        "answer_type": "mc4",
        "choices": ["A", "B", "C", "D"]
    }
}

# ========== Parameters ==========
N = 1
SEED = 42
random.seed(SEED)

models_dir = "./models"
model_files = [f for f in os.listdir(models_dir) if f.endswith(".gguf")]
results = []

# ========== Text Normalization ==========
def normalize(text):
    return text.strip().lower()

# ========== Evaluate Each Model ==========
for model_file in model_files:
    model_path = os.path.join(models_dir, model_file)
    print(f"\n=== Evaluating model: {model_file} ===")

    model = Llama(
        model_path=model_path,
        n_ctx=2048,
        n_threads=8,
        n_gpu_layers=-1
    )

    model_result = {"model": model_file, "tasks": {}}

    for task_name, config in TASKS.items():
        print(f"--- Task: {task_name} ---")

        if "subset" in config:
            dataset = load_dataset(config["dataset"], config["subset"], split=config["split"])
        else:
            dataset = load_dataset(config["dataset"], split=config["split"])

        samples = random.sample(list(dataset), N)
        correct = 0

        for sample in tqdm(samples, desc=f"Running {task_name}"):
            # Prompt construction and label
            if config["answer_type"] == "bool":
                question = sample['question']
                passage = sample['passage']
                answer = sample['answer']
                prompt = f"""Question: {question}

Passage: {passage}

Please answer TRUE or FALSE.
Answer:"""
                gold = "true" if answer else "false"

            elif config["dataset"] == "piqa":
                q = sample['goal']
                a1 = sample['sol1']
                a2 = sample['sol2']
                prompt = f"""Choose the best option for the physical question below.

Question: {q}

A: {a1}
B: {a2}

Which option is better? Answer with A or B.
Answer:"""
                gold = "a" if sample['label'] == 0 else "b"

            elif config["dataset"] == "openbookqa":
                question = sample["question_stem"]
                choices = sample["choices"]
                gold = sample["answerKey"].lower()

                prompt = f"""You are answering a multiple choice science question. Choose the correct option: A, B, C, or D.

Question: {question}

"""
                for label, text in zip(choices["label"], choices["text"]):
                    prompt += f"{label}: {text}\n"

                prompt += "\nAnswer:"

            else:
                continue

            # Inference
            response = model(
                prompt,
                max_tokens=20,
                temperature=0.7,
                stop=["\n"]
            )["choices"][0]["text"].strip().lower()

            # Prediction and comparison
            if config["answer_type"] == "bool":
                pred = "true" if "true" in response else "false" if "false" in response else ""
            elif config["answer_type"] in ["mc", "mc4"]:
                pred = None
                for opt in ['a', 'b', 'c', 'd', 'e']:
                    if opt in response:
                        pred = opt
                        break
            else:
                pred = ""

            if pred == gold:
                correct += 1

        acc = correct / N
        model_result["tasks"][task_name] = {
            "accuracy": acc,
            "correct": correct,
            "total": N
        }
        print(f"[{task_name}] Accuracy: {acc * 100:.2f}% ({correct}/{N})")

    results.append(model_result)

    # Cleanup
    del model
    gc.collect()

# ========== Save Results (Pivot Format) ==========
save_dir = "./eval"
os.makedirs(save_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"eval_results_{N}samples_{timestamp}.csv"
output_path = os.path.join(save_dir, filename)

# Flatten all results into rows for pivot
rows = []
for model_result in results:
    model_name = model_result["model"]
    for task, result in model_result["tasks"].items():
        rows.append({
            "Model": model_name,
            "Task": task,
            "Accuracy": round(result["accuracy"] * 100, 2)
        })

df = pd.DataFrame(rows)

# Pivot to "Model × Task" format
pivot_df = df.pivot(index="Model", columns="Task", values="Accuracy")

# Compute average accuracy per model
pivot_df["Avg"] = pivot_df.mean(axis=1).round(2)

# Optional: Sort by Avg
pivot_df = pivot_df.sort_values(by="Avg", ascending=False)

# Save to CSV
pivot_df.to_csv(output_path)
print(f"\n✅ Combined results saved to: {output_path}")



llama_model_loader: loaded meta data with 32 key-value pairs and 292 tensors from ./models/DeepSeek-R1-Distill-Llama-8B-Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = DeepSeek R1 Distill Llama 8B
llama_model_loader: - kv   3:                           general.basename str              = DeepSeek-R1-Distill-Llama
llama_model_loader: - kv   4:                         general.size_label str              = 8B
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                       llama.context_length u32              = 131072
llama_model_loader: -


=== Evaluating model: DeepSeek-R1-Distill-Llama-8B-Q4_0.gguf ===


init_tokenizer: initializing tokenizer for type 2
load: control token: 128254 '<|reserved_special_token_246|>' is not marked as EOG
load: control token: 128249 '<|reserved_special_token_241|>' is not marked as EOG
load: control token: 128246 '<|reserved_special_token_238|>' is not marked as EOG
load: control token: 128243 '<|reserved_special_token_235|>' is not marked as EOG
load: control token: 128242 '<|reserved_special_token_234|>' is not marked as EOG
load: control token: 128241 '<|reserved_special_token_233|>' is not marked as EOG
load: control token: 128240 '<|reserved_special_token_232|>' is not marked as EOG
load: control token: 128235 '<|reserved_special_token_227|>' is not marked as EOG
load: control token: 128231 '<|reserved_special_token_223|>' is not marked as EOG
load: control token: 128230 '<|reserved_special_token_222|>' is not marked as EOG
load: control token: 128228 '<|reserved_special_token_220|>' is not marked as EOG
load: control token: 128225 '<|reserved_special_

--- Task: BoolQ ---


Running BoolQ:   0%|          | 0/1 [00:00<?, ?it/s]llama_perf_context_print:        load time =    1845.89 ms
llama_perf_context_print: prompt eval time =    1845.75 ms /    63 tokens (   29.30 ms per token,    34.13 tokens per second)
llama_perf_context_print:        eval time =     111.60 ms /     1 runs   (  111.60 ms per token,     8.96 tokens per second)
llama_perf_context_print:       total time =    1958.60 ms /    64 tokens
Running BoolQ: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


...

--- Task: BoolQ ---


Running BoolQ:   0%|          | 0/1 [00:00<?, ?it/s]llama_perf_context_print:        load time =    8403.87 ms
llama_perf_context_print: prompt eval time =    8403.66 ms /   201 tokens (   41.81 ms per token,    23.92 tokens per second)
llama_perf_context_print:        eval time =    2175.45 ms /    19 runs   (  114.50 ms per token,     8.73 tokens per second)
llama_perf_context_print:       total time =   10589.21 ms /   220 tokens
Running BoolQ: 100%|██████████| 1/1 [00:10<00:00, 10.59s/it]


[BoolQ] Accuracy: 0.00% (0/1)
--- Task: PIQA ---


Running PIQA:   0%|          | 0/1 [00:00<?, ?it/s]Llama.generate: 1 prefix-match hit, remaining 87 prompt tokens to eval
llama_perf_context_print:        load time =    8403.87 ms
llama_perf_context_print: prompt eval time =    3432.02 ms /    87 tokens (   39.45 ms per token,    25.35 tokens per second)
llama_perf_context_print:        eval time =     115.09 ms /     1 runs   (  115.09 ms per token,     8.69 tokens per second)
llama_perf_context_print:       total time =    3548.28 ms /    88 tokens
Running PIQA: 100%|██████████| 1/1 [00:03<00:00,  3.55s/it]


[PIQA] Accuracy: 100.00% (1/1)

✅ Combined results saved to: ./eval/eval_results_1samples_20250331_194130.csv
